In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import numpy as np

In [2]:
## FIXTURE SCRAPER FROM FBREF

start_season = int(input('What is the start season?'))
current_season = int(input('What is the current season?'))


# Iterates through premier league seasons
for i in range (start_season, current_season + 1):

    # Makes sure we are in the right folder

    if not os.getcwd().endswith('Football Forecasting Version 2'):
        os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
    
    current_folder = os.getcwd()

    relative_folder = fr'{i}_{i+1}_Fixture'   

    full_path = os.path.join(current_folder, relative_folder)

    isExist = os.path.exists(full_path)

    # Creates fixture folder if it does not exist

    if not isExist:
        os.makedirs(full_path)

    url = fr'https://fbref.com/en/comps/9/{i}-{i+1}/{i}-{i+1}-Premier-League-Stats'

    # Uses user-agent to disguise

    HEADERS={'User-Agent':'Mozilla/5.0 (iPhone; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}


    # Scrapes fixture data

    r = requests.get(url, headers=HEADERS) # proxies=proxies
    print(r)
    soup = BeautifulSoup(r.text,'html.parser') 

    List_of_teams = []

    time.sleep(2)

    Team_partial_link = []
    Team_full_link = []

    table_soup = soup.find('table', {'class':'stats_table sortable min_width force_mobilize'})

    for row in table_soup.find_all('tr'):
        row_text = [e.text.strip() for e in row.find_all('td')]
        try:
            List_of_teams.append(row_text[0])
        except:
            continue

    row_team = table_soup.find_all('td',{'class':'left','data-stat':'team'})

    for e in row_team:
        a = e.find('a')
        Team_partial_link.append(e.find('a').get('href')) 

    for e in Team_partial_link:
        Team_full_link.append('https://fbref.com' + e)

    link_dictionary = dict(zip(Team_full_link,List_of_teams,))

    for url in Team_full_link:
        time.sleep(3)
        # try:
        r = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(r.text,'html.parser') 
        table_soup = soup.find('table', {'class':'stats_table sortable min_width','id':'matchlogs_for'})
        header_data_fixtures = []
        table_data_fixtures = [] 
        date = []
        new_fixtures_table_data = []

        row = table_soup.find('tr')
        header_data_fixtures = [e.text.strip() for e in row.find_all('th')]

        for row in table_soup.find_all('tr'):
            date_text = [e.text.strip() for e in row.find_all('th',{'class':'left'})]
            date.append(date_text)
            row_text = [e.text.strip() for e in row.find_all('td')]
            table_data_fixtures.append(row_text)

        new_fixture_table_data = []
        for x in list(zip(date,table_data_fixtures))[1:]:
            fixture_row = []
            for j in list(x):
                for k in j:
                    fixture_row.append(k)
            new_fixtures_table_data.append(fixture_row)

        fixtures_table = pd.DataFrame(new_fixtures_table_data, columns = header_data_fixtures )
        fixtures_table = fixtures_table.replace({'': np.nan })
        fixtures_table = fixtures_table[fixtures_table['Comp'] == 'Premier League']
        fixtures_table = fixtures_table.drop(columns = ['Time','Comp','Round','Day','Attendance','Captain','Formation','Referee','Match Report','Notes'])
        fixtures_table['Club'] = link_dictionary[url]
        fixtures_table['Date'] = pd.to_datetime(fixtures_table ['Date'])

    
        # Saves fixture table as csv
        fixtures_table.to_csv(fr'{full_path}\{i}_{i+1}_{link_dictionary[url]}.csv')


ValueError: invalid literal for int() with base 10: ''